## Import important libraries

In [364]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

## Create a Class for Batching
This class determines how the data is splitted into batches and fed to the algorithm

In [358]:


class Audiobooks_Data_Reader():
    #This function loads the initial batch from the .npz files 
    def __init__(self, dataset, batch_size = None):
    
        npz = np.load('audiobook_data_{0}.npz' .format(dataset))
        #define inputs and targets and data types
        self.inputs = npz['inputs'].astype(np.float)
        self.targets = npz['targets'].astype(np.int)

        # write a while loop which counts the batch number, determined by the batch size
        # when validating or testing we take the whole data in a single batch in that cass batch_size = None
        if batch_size == None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.current_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
    
    # A function which loads the next batches
    def __next__(self):
        if self.current_batch >= self.batch_count:
            self.current_batch = 0
            raise StopIteration()
        
        batch_slice = slice(self.current_batch * self.batch_size, (self.current_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.current_batch += 1
        
        # One-hot encoding the targets
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        return inputs_batch, targets_one_hot
    
    def __iter__(self):
        return self
    

## Outlining the model

In [359]:
#declare the shape outline of model
input_size = 10
output_size = 2
hidden_layer_size = 30

tf.reset_default_graph()

# declare the placeholders
inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.int32, [None, output_size])

#create the first hidden layer
# Activation function is Leaky_RELU
weight_1 = tf.get_variable('weight_1', [input_size,hidden_layer_size])
biases_1 = tf.get_variable('biases_1', [hidden_layer_size])
output_1 = tf.nn.leaky_relu(tf.matmul(inputs, weight_1) + biases_1)

#create second hidden layer
# Activation function is Leaky_RELU
weight_2 = tf.get_variable('weight_2', [hidden_layer_size,  hidden_layer_size])
biases_2 = tf.get_variable('biases_2', [hidden_layer_size])
output_2 = tf.nn.leaky_relu(tf.matmul(output_1, weight_2) + biases_2)

#create third hidden layer
# Activation function is Leaky_RELU
weight_3 = tf.get_variable('weight_3', [hidden_layer_size,  hidden_layer_size])
biases_3 = tf.get_variable('biases_3', [hidden_layer_size])
output_3 = tf.nn.leaky_relu(tf.matmul(output_2, weight_3) + biases_3)

#create fourth hidden layer
# Activation function is Leaky_RELU
weight_4 = tf.get_variable('weight_4', [hidden_layer_size,  hidden_layer_size])
biases_4 = tf.get_variable('biases_4', [hidden_layer_size])
output_4 = tf.nn.leaky_relu(tf.matmul(output_3, weight_4) + biases_4)

#create fifth hidden layer
# Activation function is Leaky_RELU
weight_5 = tf.get_variable('weight_5', [hidden_layer_size,  hidden_layer_size])
biases_5 = tf.get_variable('biases_5', [hidden_layer_size])
output_5 = tf.nn.leaky_relu(tf.matmul(output_4, weight_5) + biases_5)

# lets create output layer
# Activation function is Leaky_RELU
weight_6 = tf.get_variable('weight_6', [hidden_layer_size,  output_size])
biases_6 = tf.get_variable('biases_6', [output_size])
outputs = tf.matmul(output_5, weight_6) + biases_6

#apply activatiotion function and calculate objective function of the final layer
loss =tf.nn.softmax_cross_entropy_with_logits(logits = outputs, labels = targets)
# reduce loss to the mean. this boasts performance (tf.reduce_mean() is a func to  find the mean of atensor)
mean_loss = tf.reduce_mean(loss)

#Choose the Optimization function and implement Batching
optimize = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(mean_loss)

#calculate the accuracy
#fist check if the outputs equals the targets
out_equals_target = tf.equal(tf.argmax(outputs, axis =1), tf.argmax(targets, axis=1))


# the accuracy is the mean of the out_equals_target tensor
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)

# set the batch size and early stopping

batch_size = 100

max_epochs = 50
prev_validation_loss = 9999999.

# Loading the data using the Audiobooks_Data_Reader class
train_data = Audiobooks_Data_Reader('train', batch_size )
validation_data = Audiobooks_Data_Reader('validation')



C:\Users\eugen\AppData\Local\Temp\ipykernel_11144\3946767040.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.inputs = npz['inputs'].astype(np.float)
C:\Users\eugen\AppData\Local\Temp\ipykernel_11144\3946767040.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/

## Creating loop to Train and validate the model

In [360]:
for epoch_counter in range(max_epochs):
    
    curr_epoch_loss = 0.
    
    for input_batch, target_batch in train_data:
        _, batch_loss = sess.run([optimize, mean_loss], 
            feed_dict={inputs: input_batch, targets: target_batch})
        
        curr_epoch_loss += batch_loss
        
    curr_epoch_loss /= train_data.batch_count
    
    validation_loss = 0.
    validation_accuracy = 0.
    
    for input_batch, target_batch in validation_data:
        validation_loss, validation_accuracy = sess.run([mean_loss, accuracy], 
        feed_dict={inputs: input_batch, targets: target_batch})   
        
    print('Epoch '+str(epoch_counter+1)+
          '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss)+
          '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
          '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%')
    
    if validation_loss > prev_validation_loss:
        break
        
    prev_validation_loss = validation_loss
    
print('End of training.')

Epoch 1. Training loss: 0.616. Validation loss: 0.533. Validation accuracy: 70.69%
Epoch 2. Training loss: 0.459. Validation loss: 0.401. Validation accuracy: 80.54%
Epoch 3. Training loss: 0.376. Validation loss: 0.367. Validation accuracy: 81.66%
Epoch 4. Training loss: 0.358. Validation loss: 0.353. Validation accuracy: 81.66%
Epoch 5. Training loss: 0.353. Validation loss: 0.339. Validation accuracy: 81.88%
Epoch 6. Training loss: 0.347. Validation loss: 0.332. Validation accuracy: 82.55%
Epoch 7. Training loss: 0.341. Validation loss: 0.327. Validation accuracy: 82.33%
Epoch 8. Training loss: 0.337. Validation loss: 0.324. Validation accuracy: 82.10%
Epoch 9. Training loss: 0.335. Validation loss: 0.321. Validation accuracy: 82.55%
Epoch 10. Training loss: 0.333. Validation loss: 0.319. Validation accuracy: 82.10%
Epoch 11. Training loss: 0.331. Validation loss: 0.317. Validation accuracy: 82.33%
Epoch 12. Training loss: 0.329. Validation loss: 0.316. Validation accuracy: 82.77%
E

## Testing the Model

In [362]:
#to test the model we forward propagate with the test data
test_data = Audiobooks_Data_Reader('test')
for input_batch, target_batch in test_data:
       test_accuracy = sess.run([accuracy], 
        feed_dict={inputs: input_batch, targets: target_batch}) 
        
test_accuracy_percent = test_accuracy[0] * 100.
# Print the test accuracy
print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')

Test accuracy: 82.81%


C:\Users\eugen\AppData\Local\Temp\ipykernel_11144\3946767040.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.inputs = npz['inputs'].astype(np.float)
C:\Users\eugen\AppData\Local\Temp\ipykernel_11144\3946767040.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/

## Conclusion
In conclusion, the machine learning algorithm developed to predict whether an Audio book customer will buy again has shown promising results, achieving a text accuracy of 82.81%. While there is room for improvement, this algorithm has the potential to help businesses better understand their customers' behavior and preferences, allowing them to tailor their marketing strategies accordingly. With further refinement and application, this algorithm has the potential to be a valuable tool for businesses seeking to enhance customer retention and drive revenue growth. 

Note: Code resue for official purpose requires prior permission from Eugene  Oboh.